In [ ]:
# !pip install datasets transformers scikit-learn evaluate torchinfo tqdm accelerate -U

# Access Token 생성
- 학습된 모델을 Huggingface hub에 올리기 위해서는 access token이 필요하다.

![huggingface_create_apikey.png](figures/huggingface_accesstoken.png)
![huggingface_create_apikey.png](figures/huggingface_accesstoken2.png)

- 1. 로그인 -> 2. Profile -> 3. Access Tokens 선택
- 생성할 때 `write` 권한을 선택한다.

# HuggingFace Datasets 패키지

- HuggingFace Datasets는 머신러닝 모델 학습에 필요한 대규모 데이터셋을 효율적으로 로드, 전처리, 변환, 스트리밍할 수 있도록 지원하는 라이브러리이다.
- HuggingFace Hub에 등록된 다양한 데이터셋을 동일한 인터페이스로 접근할 수 있다.
- 설치

  - `pip install datasets`
- 공식 문서

  - [Dataset Hub: https://huggingface.co/datasets](https://huggingface.co/datasets)
  - [https://github.com/huggingface/datasets](https://github.com/huggingface/datasets)
  - [https://huggingface.co/docs/datasets/index](https://huggingface.co/docs/datasets/index)
## Dataset과 DatasetDict

### Dataset

- 하나의 데이터셋을 테이블 형태(행 = 샘플, 열 = Feature)로 저장하는 구조.
- 텍스트, 이미지, 오디오, 멀티모달 데이터 지원.
- PyTorch, TensorFlow 학습 파이프라인과 연동 가능.
- Lazy Loading 및 Streaming을 통해 대규모 데이터도 메모리 효율적으로 처리.

### DatasetDict

- 여러 Dataset을 key-value 형태로 묶어 관리한다.

  - 예: `{"train": Dataset, "validation": Dataset, "test": Dataset}`
- key를 통해 데이터셋에 접근.

## DatasetDict / Dataset 로딩
- `datasets.load_dataset()` 이용
  
### Hub 데이터 로딩
- Huggingface hub에 등록된 Dataset의 id를 전달하면  다운로드 후 로딩한다.

```python
from datasets import load_dataset
dataset_dict = load_dataset("imdb")
```
![img](figures/huggingface_dataset.png)

### 로컬 파일 로딩

```python
dataset_dict = load_dataset(
    "csv",
    data_files={
        "train": "data/train.csv",
        "validation": "data/valid.csv",
        "test": "data/test.csv"
    }
)
```

### 다양한 소스에서 Dataset 생성

```python
from datasets import Dataset

Dataset.from_pandas(df) # DataFrame을 Dataset으로 로딩
Dataset.from_dict({"text": [...], "label": [...]}) # 딕셔너리를 Dataset으로 로딩
Dataset.from_csv("path.csv") # 경로의 csv파일을 Dataset으로 로딩
```

## Dataset 주요 기능

### 데이터 접근(조회)

- 인덱싱으로 샘플 조회: `dataset[0]`
- 슬라이싱: `dataset[:10]`

### 전처리 관련 메소드

#### map()

데이터셋의 모든 데이터(또는 지정한 batch)에 변환 함수를 적용한다.

```python
dataset.map(
    function,                    # 각 샘플/배치에 적용할 함수
    batched=False,               # True: 배치 단위 입력
    batch_size=1000,             # batched=True일 때 배치사이즈. 기본값: 1000
    remove_columns=None,         # 변환 후 제거할 컬럼 리스트
    num_proc=None,               # 병렬 처리 프로세스 개수
)
```

##### map에 전달하는 함수(function)의 입력(parameter)/출력(return) 형태
- parameter: dict - Dataset의 데이터를 dict(컬럼명, 값) 로 받는다. batched=False일 경우 개별 값, True일 경우 값은 iterable
- return: dict - 처리한 결과. dict(컬럼명, 처리값) 지정한 컬럼에 처리한 값을 저장한다. 컬럼명이 **기존 컬럼이면 덮어쓰고, 없는 컬럼이면 추가**한다. `remove_columns=컬럼명` 옵션을 지정하면 처리가 끝난 뒤 지정한 컬럼을 제거할  수 있다.
- `batched=False` (기본값) 일 때

  - 함수 입력: dict (하나의 row)
  - 함수 반환: dict (추가될 컬럼 포함)

    ```python
    def add_length(example):
        example["text_len"] = len(example["text"])
        return example

    dataset = dataset.map(add_length)
    ```

- `batched=True` 일때 (배치단위 입력)

  - 함수 입력: dict (여러 row의 컬럼이 리스트로 묶여 들어옴)
  - 함수 반환: dict (리스트 형태 유지)

    ```python
    def add_length_batch(batch):
        return {"text_len": [len(t) for t in batch["text"]]}

    dataset = dataset.map(add_length_batch, batched=True)
    ```

#### filter()

조건 함수를 적용하여 True를 반환하는 row만 남긴다.

```python
dataset.filter(
    function,           # 조건 함수
    batched=False,      # 배치 처리 여부
    num_proc=None,      # 병렬 처리
    desc=None
)
```

##### filter()에 전달하는 함수의 입력(parameter)/출력(return) 형태
- parameter: dict - Dataset의 데이터를 dict(컬럼명, 값) 로 받는다. batched=False일 경우 개별 값, True일 경우 값은 iterable
- return: bool -  Dataset에 남길지 말지 조건
- `batched=False`: 한 줄씩 조건 적용

  ```python
  dataset = dataset.filter(lambda x: x["label"] == 1)
  ```

- `batched=True`: 배치 활용

  ```python
  def filter_positive(batch):
      return [label == 1 for label in batch["label"]]

  dataset = dataset.filter(filter_positive, batched=True)
  ```

## 기타 유용 메소드

- **컬럼 삭제**
  - `dataset = dataset.remove_columns(["column_name"])`
- **컬럼 이름 변경**
  - `dataset = dataset.rename_column("orig", "new")`
- **데이터 분할**
  - `dataset.train_test_split(test_size=0.2)`

- **저장**

  ```python
  dataset.save_to_disk("path/")
  dataset = datasets.load_from_disk("path/")
  ```

- 모델학습 프레임워크용 포맷 변환
  - 기본적으로 Dataset의 데이터들은 list 로 저장됨.
  - pytorch나 tensorflow에서 사용할 수있도록 format을 변환. (값을 제공할 때 지정한 타입으로 변환해서 반환한다.)

    ```python
    dataset.set_format(type="torch") # numpy, torch, tensorflow, python
    ```

- Streaming 로딩
    - Huggingface Hub의 데이터셋이 너무 클 경우(수백 GB) 필요한 데이터만 순차적으로 가져와 로딩하여 처리할 수있다.
      ```python
      ds_stream = load_dataset("c4", split="train", streaming=True)

      for sample in ds_stream:
        sample 로 사용
      ```


# 유튜브 댓글 감정 분석 데이터셋
- https://huggingface.co/datasets/LLM-SocialMedia/Korean-YouTube-Comment-Sentiment-Dataset
- `LLM-SocialMedia/Korean-YouTube-Comment-Sentiment-Dataset`

## Train/Test set split

## 모델, 토크나이저 loading

- 모델 별 Model 클래스를 이용하거나 Auto class를 이용해 모델, 전처리기(tokenizer, ImageProcessor 등)을 로딩한다.
    - Huggingface에 저장된 model name을 입력해서 pretrained 모델을 loading 한다.
    - fine tuning 한 경우 모델 저장 디렉토리 경로를 넣어 pretrained 모델을 loading한다.
- AutoModel은 model name을 주면 그 모델이 학습한 base 모델에 맞는 객체를 생성해서 반환한다.
    - Auto Model은 task 별로 다양한 클래스들이 있다.
        - 클래스 이름 형식: AutoModelFor{Task형식}
        - ex) `AutoModelForObjectDetection`, `AutoModelForSequenceClassification`
    - https://huggingface.co/docs/transformers/model_doc/auto
    - 전처리기(tokenzier)는 사용하려는 모델이 사용한 전처리기를 사용해야 한다.

## 학습 (Hugging Face Transformers)

- Transformers 라이브러리는 모델 학습을 위해 `TrainingArguments`, `Trainer` 클래스를 제공한다.
- `Trainer`:
  -  데이터 로딩, 학습 루프, 평가, 로깅, 모델 저장 등을 자동으로 처리하여, 복잡한 학습 절차를 단순한 코드로 구성할 수 있도록 돕는다.
- `TrainingArguments`:
  -  `Trainer`가 사용할 **학습 관련 설정**(학습 파라미터, 로그/체크포인트 저장 방식 등)을 담당하는 클래스이다.
- 이들을 활용하면 **학습 옵션 설정, 로깅, Gradient Accumulation, Mixed Precision(AMP) 적용, 평가 및 체크포인트 관리** 등을 손쉽게 수행할 수 있다.
- `Trainer`를 사용하려면 **Datasets** 라이브러리 또는 `torch.utils.data.Dataset` 형식의 데이터셋이 필요하다.
- 일반적으로는 `Trainer`만 써도 파인튜닝을 빠르고 편리하게 할 수 있다. 하지만 학습 과정에서 특별한 계산을 해야 하거나, 일반적이지 않은 방식으로 학습을 진행해야 할 때는 `Trainer`를 쓰기 어렵다. 이럴 때는 직접 `PyTorch`로 학습 루프를 작성해 원하는 방식대로 조정한다.


### 평가함수 정의
- `evaluate` 패키지를 이용해 정의한다.
- `evaluate` 패키지
  - evaluate 패키지는 **모델 성능을 측정하기 위한 평가 지표(Metrics)**를 손쉽게 불러오고 사용할 수 있도록 하는 라이브러리이다.
  - 다양한 머신러닝 분야에서 사용되는 평가지표를 지원하며 필요하면 사용저 정의 평가지표를 정의할 수도 있다.

### Collator 함수

- **Collator(Data Collator)**는 **데이터셋에서 샘플 단위(raw examples)를 꺼낸 뒤 → 하나의 배치(batch)로 묶을 때 필요한 전처리를 수행하는 함수**이다.
- 즉, DataLoader가 `batch_size=8`로 8개의 샘플을 가져왔다면, collator는 이 8개의 샘플을 **모델이 바로 입력할 수 있는 형태**로 변환한다.
- Transformer 모델은 각 샘플의 길이가 다르므로, 이들을 자동으로 **패딩(padding)**·**마스크 생성**·**텐서 변환** 등을 해주는 장치가 필요하며, 이 역할을 담당하는 것이 **collator**이다.

#### collator가 수행하는 핵심 역할
- **정적 패딩(Static padding)/동적 패딩(Dynamic Padding)**
  - 모든 문장의 길이를 지정한 길이에 맞추기 위해 길이가 적은 경우 패딩 토큰을 추가한다.
  - 모든 학습 데이터셋의 길이를 동일하게 맞추는 것을 정적패딩, 배치 단위로 맞추는 것을 동적 패딩이라고 한다.
  - 전체 데이터셋을 미리 패딩하는 정적 패딩보다 **동적 패딩**이 메모리를 절약하고 속도를 증가시켜 더 효율적이다.

- **Attention Mask 생성**
  - 패딩된 부분은 계산에서 제외해야 한다. collator는 `attention_mask`를 자동으로 생성해 준다.

    ```bash
    Input Ids:     [101, 2003, 2023, 102, 0,   0]
    Attention Mask:[ 1,   1,   1,   1, 0,   0]
    ```

- **Label 패딩 / 정렬**

  - 텍스트 생성/요약 등에서는 target sequence도 padding 필요
  - BERT의 MLM(Masked LM)에서는 label에서 패딩에 -100을 넣는 등 규칙이 있음  (PyTorch CrossEntropyLoss는 -100을 무시)

- **텐서 변환**

    - 리스트(list) 형태의 샘플을 PyTorch/TensorFlow 텐서로 변환한다.

#### Hugging Face에서 제공하는 대표 Collator들

| Collator                             | 주 사용 모델         | 역할                      |
| ------------------------------------ | --------------- | ----------------------- |
| `DataCollatorWithPadding`            | 대부분의 모델         | 동적 패딩 처리                |
| `DataCollatorForLanguageModeling`    | BERT, RoBERTa 등 | MLM mask 생성 및 label 생성  |
| `DataCollatorForSeq2Seq`             | T5, BART 등      | 입력/출력 둘 다 패딩 및 label 정리 |
| `DataCollatorForTokenClassification` | NER 등           | token-level label 패딩    |
| `DefaultDataCollator`                | 간단한 task        | 미리 패딩된 입력만 병합           |

#### 예시

```python
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

collator = DataCollatorWithPadding(tokenizer=tokenizer)

# DataLoader 예시
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=4, collate_fn=collator)
```


# 추론